In [13]:
import struct
import json 

In [14]:
class Packet:
    def __init__(self, timestamp, length, bytearray, id=None, type=None):
        self.timestamp = timestamp
        self.length = length
        self.bytearray = bytearray
        self.id = id
        self.type = type
    def __repr__(self) -> str:
        return f'Packet of type: {self.type} with id {self.id} starting at {self.timestamp} with a bytearray {self.bytearray} of length {self.length}'

In [15]:
packets = []
#open byte file
#read in packet data
#structure: timestamp(integer) length(integer) packet(bytearray)
with open("./data/2021_05_02_22_04_36_1/recording.tmcpr", "rb") as f:
    while timestamp := f.read(4):
        timestamp = struct.unpack('>i', timestamp)[0]
        length = struct.unpack('>i', f.read(4))[0]
        bytearray = f.read(length)
        packets.append(
            Packet(
                timestamp=timestamp,
                length=length,
                bytearray=bytearray
            )
        )

In [16]:
#skip first packet because its a login success packet
packets = packets[1:]

In [17]:
#read the protocol for 1.8
#Protocol 47 
#JSON from https://github.com/PrismarineJS/minecraft-data/blob/master/data/pc/1.8/protocol.json
protocol = json.load(open('./data/protocol.json', 'r'))

In [18]:
#function to read specific type of integer from minecraft
#variable length integer 
def read_var_int(b):
    byte = b.read(1)
    if not byte:
        return None
    value = ord(byte)
    if value <= 127:
        return value
    value, shift, quantum = value & 0x7f, 7, value
    while 1:
        byte = b.read(1)
        if not byte:
            return None
        quantum = ord(byte)
        value, shift = value + ((quantum & 0x7f) << shift), shift + 7
        if not quantum & 0x80:
            break
    return value

In [19]:
#json dict of packet name and the make up of the packet
packet_json = protocol['play']['toClient']['types']

In [20]:
#creates a mapping between id numbers and names of packets

#gets the mappings from the protocol
packet_types = packet_json['packet'][1][0]['type'][1]['mappings']

#adds packet_ in front of it to work with the keys of packet_json
packet_types_values = ["packet_"+x for x in list(packet_types.values())]

#converts the hex string ids to numbers
packet_types_keys = [int(x, 16) for x in list(packet_types.keys())]

#creates a dictionary of them
packet_types = dict(zip(packet_types_keys, packet_types_values))

In [21]:
#possible functions needed to read data 
def read_entity_meta_data():
    pass
def read_slot_data():
    pass
def read_NBT_data():
    pass
def read_chunk_data():
    pass

In [22]:
#classifies the packet and removes the id byte and modifies length accordingly
for i, packet in enumerate(packets):
    packet.type = packet_types[packet.bytearray[0]]
    packet.id = packet.bytearray[0]
    packet.bytearray = packet.bytearray[1:]
    packet.length-=1